<a href="https://colab.research.google.com/github/gibsonx/CE888/blob/master/Federated_Minst.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
#Modified by Gibson Xue on July 25, 2021
!pip uninstall --yes tensorflow tensorboard tb-nightly
!pip install --quiet tensorflow==2.5.0
!pip install --quiet tensorflow-federated==0.19.0
!pip install --quiet nest-asyncio
!pip install --quiet tensorboard  # or tensorboard, but not both


Uninstalling tensorflow-2.5.0:
  Successfully uninstalled tensorflow-2.5.0
Uninstalling tensorboard-2.5.0:
  Successfully uninstalled tensorboard-2.5.0
     |████████████████████████████████| 454.3MB 33kB/s 
     |████████████████████████████████| 6.0MB 34.4MB/s 
     |████████████████████████████████| 604kB 4.9MB/s 
     |████████████████████████████████| 194kB 37.4MB/s 
     |████████████████████████████████| 174kB 40.4MB/s 
     |████████████████████████████████| 51kB 5.4MB/s 
ERROR: spacy 2.2.4 has requirement tqdm<5.0.0,>=4.38.0, but you'll have tqdm 4.28.1 which is incompatible.
ERROR: pymc3 3.11.2 has requirement cachetools>=4.2.1, but you'll have cachetools 3.1.1 which is incompatible.
ERROR: fbprophet 0.7.1 has requirement tqdm>=4.36.1, but you'll have tqdm 4.28.1 which is incompatible.
ERROR: datascience 0.10.6 has requirement folium==0.2.1, but you'll have folium 0.8.3 which is incompatible.


In [53]:
import tensorflow as tf
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
%matplotlib inline
import nest_asyncio
nest_asyncio.apply()
print(tf.__version__)

2.5.0


In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [54]:
import collections
import functools
import os
import time
from tensorflow.keras.datasets import mnist
import numpy as np
import tensorflow as tf
import tensorflow_federated as tff
import tensorflow_datasets as tfds
import pandas as pd
np.random.seed(0)

from tensorflow import reshape, nest, config
from tensorflow.keras import losses, metrics, optimizers
import tensorflow_federated as tff
from matplotlib import pyplot as plt
from pathlib import Path
from sklearn.model_selection import train_test_split
# Test the TFF is working:
tff.federated_computation(lambda: 'Hello, World!')()
from tensorflow.keras import Sequential
from tensorflow.keras.layers import Dense, Dropout, Flatten, Conv2D, MaxPool2D

In [55]:
def create_keras_model():
    model = Sequential([
        # Conv2D(32, (3, 3), activation='relu', input_shape=(28, 28, 1)),
        # MaxPool2D(pool_size=(2, 2)),
        # Conv2D(64, (3, 3), activation='relu'),
        # MaxPool2D(pool_size=(2, 2)),
        # Flatten(),
        # Dense(512, activation='relu'),
        # Dense(10, activation='softmax')
        Conv2D(filters = 64, kernel_size = (5,5),padding = 'Same',activation ='relu', input_shape = (28,28,1)),
        Conv2D(filters = 64, kernel_size = (5,5),padding = 'Same',activation ='relu'),
        MaxPool2D(pool_size=(2,2)),
        Dropout(0.25),
        Conv2D(filters = 128, kernel_size = (3,3),padding = 'Same',activation ='relu'),
        Conv2D(filters = 128, kernel_size = (3,3),padding = 'Same',activation ='relu'),
        MaxPool2D(pool_size=(2,2), strides=(2,2)),
        Dropout(0.3),
        Flatten(),
        Dense(512, activation = "relu", use_bias= True),
        Dropout(0.5),
        Dense(10, activation = "softmax")
    ])
    return model

In [87]:
experiment_name = "mnist"
method = "tff_training"
client_lr = 1e-2
server_lr = 1e-2
split = 4
NUM_ROUNDS = 5
NUM_EPOCHS = 5
BATCH_SIZE = 20
PREFETCH_BUFFER = 10

df_orig_train = pd.read_csv('/content/drive/MyDrive/mnist_train.csv')
df_orig_test = pd.read_csv('/content/drive/MyDrive/mnist_test.csv')

# X_train = df_orig_train.drop('label',axis=1).to_numpy().astype(np.int32).reshape(59999,28,28,1)
# Y_train = df_orig_train['label'].to_numpy().astype(np.int32)

x_train = df_orig_train.iloc[:,1:].to_numpy().astype(np.float32).reshape(59999,28,28,1)[:3000]
y_train = df_orig_train.iloc[:,0].to_numpy().astype(np.int32).reshape(59999,1)[:3000]
x_test = df_orig_test.iloc[:,1:].to_numpy().astype(np.float32).reshape(9999,28,28,1)[:500]
y_test = df_orig_test.iloc[:,0].to_numpy().astype(np.int32).reshape(9999,1)[:500]


# (x_train, y_train), (x_test, y_test) = mnist.load_data()

# x_train = x_train.astype(np.float32).reshape(60000, 28, 28, 1)[:300]
# y_train = y_train.astype(np.int32).reshape(60000, 1)[:300]
# x_test = x_test.astype(np.float32).reshape(10000, 28, 28, 1)[:300]
# y_test = y_test.astype(np.int32).reshape(10000, 1)[:300]

In [88]:
this_dir = Path.cwd()
model_dir = this_dir / "saved_models" / experiment_name / method
output_dir = this_dir / "results" / experiment_name / method

if not model_dir.exists():
    model_dir.mkdir(parents=True)

if not output_dir.exists():
    output_dir.mkdir(parents=True)

In [89]:
total_image_count = len(x_train)
image_per_set = int(np.floor(total_image_count/split))

client_train_dataset = collections.OrderedDict()
for i in range(1, split+1):
    client_name = "client_" + str(i)
    start = image_per_set * (i-1)
    end = image_per_set * i

    print(f"Adding data from {start} to {end} for client : {client_name}")
    data = collections.OrderedDict((('label', y_train[start:end]), ('pixels', x_train[start:end])))
    client_train_dataset[client_name] = data

train_dataset = tff.simulation.datasets.TestClientData(client_train_dataset)

sample_dataset = train_dataset.create_tf_dataset_for_client(train_dataset.client_ids[0])
sample_element = next(iter(sample_dataset))

SHUFFLE_BUFFER = image_per_set


def preprocess(dataset):

  def batch_format_fn(element):
    """Flatten a batch `pixels` and return the features as an `OrderedDict`."""
    print(element['pixels'])
    return collections.OrderedDict(
        x=reshape(element['pixels'], [-1, 28, 28, 1]),
        y=reshape(element['label'], [-1, 1]))

  return dataset.repeat(NUM_EPOCHS).shuffle(SHUFFLE_BUFFER).batch(
      BATCH_SIZE).map(batch_format_fn).prefetch(PREFETCH_BUFFER)

preprocessed_sample_dataset = preprocess(sample_dataset)
sample_batch = nest.map_structure(lambda x: x.numpy(), next(iter(preprocessed_sample_dataset)))

def make_federated_data(client_data, client_ids):
    return [preprocess(client_data.create_tf_dataset_for_client(x)) for x in client_ids]

federated_train_data = make_federated_data(train_dataset, train_dataset.client_ids)
print('Number of client datasets: {l}'.format(l=len(federated_train_data)))
print('First dataset: {d}'.format(d=federated_train_data[0]))
federated_train_data


Adding data from 0 to 750 for client : client_1
Adding data from 750 to 1500 for client : client_2
Adding data from 1500 to 2250 for client : client_3
Adding data from 2250 to 3000 for client : client_4
Tensor("args_1:0", shape=(None, 28, 28, 1), dtype=float32)
Tensor("args_1:0", shape=(None, 28, 28, 1), dtype=float32)
Tensor("args_1:0", shape=(None, 28, 28, 1), dtype=float32)
Tensor("args_1:0", shape=(None, 28, 28, 1), dtype=float32)
Tensor("args_1:0", shape=(None, 28, 28, 1), dtype=float32)
Number of client datasets: 4
First dataset: <PrefetchDataset shapes: OrderedDict([(x, (None, 28, 28, 1)), (y, (None, 1))]), types: OrderedDict([(x, tf.float32), (y, tf.int32)])>


[<PrefetchDataset shapes: OrderedDict([(x, (None, 28, 28, 1)), (y, (None, 1))]), types: OrderedDict([(x, tf.float32), (y, tf.int32)])>,
 <PrefetchDataset shapes: OrderedDict([(x, (None, 28, 28, 1)), (y, (None, 1))]), types: OrderedDict([(x, tf.float32), (y, tf.int32)])>,
 <PrefetchDataset shapes: OrderedDict([(x, (None, 28, 28, 1)), (y, (None, 1))]), types: OrderedDict([(x, tf.float32), (y, tf.int32)])>,
 <PrefetchDataset shapes: OrderedDict([(x, (None, 28, 28, 1)), (y, (None, 1))]), types: OrderedDict([(x, tf.float32), (y, tf.int32)])>]

In [90]:
def model_fn():
  # We _must_ create a new model here, and _not_ capture it from an external
  # scope. TFF will call this within different graph contexts.

  keras_model = create_keras_model()
  return tff.learning.from_keras_model(
      keras_model,
      input_spec=preprocessed_sample_dataset.element_spec,
      loss=losses.SparseCategoricalCrossentropy(),
      metrics=[metrics.SparseCategoricalAccuracy()])


iterative_process = tff.learning.build_federated_averaging_process(
    model_fn,
    client_optimizer_fn=lambda: optimizers.Adam(learning_rate=client_lr),
    server_optimizer_fn=lambda: optimizers.SGD(learning_rate=server_lr))

print(str(iterative_process.initialize.type_signature))

( -> <model=<trainable=<float32[5,5,1,64],float32[64],float32[5,5,64,64],float32[64],float32[3,3,64,128],float32[128],float32[3,3,128,128],float32[128],float32[6272,512],float32[512],float32[512,10],float32[10]>,non_trainable=<>>,optimizer_state=<int64>,delta_aggregate_state=<value_sum_process=<>,weight_sum_process=<>>,model_broadcast_state=<>>@SERVER)


In [91]:
tff_train_acc = []
tff_val_acc = []
tff_train_loss = []
tff_val_loss = []

eval_model = None

In [92]:
# state.model.assign_weights_to(eval_model)
state = iterative_process.initialize()
# eval_model = create_keras_model()
# eval_model.compile(optimizer=optimizers.Adam(learning_rate=client_lr),
#                     loss=losses.SparseCategoricalCrossentropy(),
#                     metrics=[metrics.SparseCategoricalAccuracy()])

eval_model = None
for round_num in range(1, NUM_ROUNDS+1):
    state, tff_metrics = iterative_process.next(state, federated_train_data)
    eval_model = create_keras_model()
    eval_model.compile(optimizer=optimizers.Adam(learning_rate=client_lr),
                       loss=losses.SparseCategoricalCrossentropy(),
                       metrics=[metrics.SparseCategoricalAccuracy()])

    state.model.assign_weights_to(eval_model)

    ev_result = eval_model.evaluate(x_test, y_test, verbose=0)
    print('round {:2d}, metrics={}'.format(round_num, tff_metrics))
    print(f"Eval loss : {ev_result[0]} and Eval accuracy : {ev_result[1]}")


if eval_model:
    eval_model.save(model_dir / (experiment_name + ".h5"))
else:
    print("training didn't started")
    exit()                    

round  1, metrics=OrderedDict([('broadcast', ()), ('aggregation', OrderedDict([('mean_value', ()), ('mean_weight', ())])), ('train', OrderedDict([('sparse_categorical_accuracy', 0.3166), ('loss', 2.071793)])), ('stat', OrderedDict([('num_examples', 15000)]))])
Eval loss : 6.67402982711792 and Eval accuracy : 0.1679999977350235
round  2, metrics=OrderedDict([('broadcast', ()), ('aggregation', OrderedDict([('mean_value', ()), ('mean_weight', ())])), ('train', OrderedDict([('sparse_categorical_accuracy', 0.37073332), ('loss', 1.9195226)])), ('stat', OrderedDict([('num_examples', 15000)]))])
Eval loss : 3.1638567447662354 and Eval accuracy : 0.17000000178813934
round  3, metrics=OrderedDict([('broadcast', ()), ('aggregation', OrderedDict([('mean_value', ()), ('mean_weight', ())])), ('train', OrderedDict([('sparse_categorical_accuracy', 0.442), ('loss', 1.731934)])), ('stat', OrderedDict([('num_examples', 15000)]))])
Eval loss : 2.184492349624634 and Eval accuracy : 0.23399999737739563
roun